In [1]:
from openpyxl import load_workbook, Workbook
filename = r"C:/Users/twish/Documents/wells fargo/z-spread_calculator.xlsx"
wb = load_workbook(filename)
ws = wb['Sheet1']

govt_bond_yield = []
corporate_bond_cash_flow = []
face_value = 100

for index, row in enumerate(ws.iter_rows()):
	if row[0].value == 'Yield':
		for sem in range(1,11):
			if row[sem].value is not None:
				govt_bond_yield.append(float(row[sem].value))
	if row[0].value =='Face':
		face_value = row[1].value
	if row[0].value == 'Coupon':
		coupon = row[1].value
		market_price = face_value*(1-coupon)
	if row[0].value == 'Cash Flow':
		for sem in range(1,11):
			if row[sem].value is not None:
				corporate_bond_cash_flow.append(float(row[sem].value))

def discounted_value(amount, rate, periods):
	return amount/(1+rate)**periods if amount > 0 else 0
print(corporate_bond_cash_flow)

def compute_dcf(govt_bond_yield,corporate_bond_cash_flow, zspread):
	DCF = [] # discounted cash flow for no z-spread case
	n_cycles = len(corporate_bond_cash_flow)
	for n in range(0,n_cycles):
	    rate = (govt_bond_yield[n]+zspread)/2
	    cash_flow = corporate_bond_cash_flow[n]
	    dcf = discounted_value(cash_flow,rate, n+1)
	    DCF.append(dcf)
	return DCF
DCF = compute_dcf(govt_bond_yield,corporate_bond_cash_flow,0)
dcf_value = sum(DCF)
print(DCF, dcf_value)

alpha = 0.01
initial_value = 0.0
init_dir = 1.0 # -1 for negative
fn = dcf_value - market_price
c = 0
while abs(fn) >= 0.01:
	c = c + 1
	new_value = initial_value + alpha*init_dir
	new_cash_flow = compute_dcf(govt_bond_yield,corporate_bond_cash_flow,new_value)
	new_fn = sum(new_cash_flow) - market_price
	new_dir = new_fn/abs(new_fn)
	if not (init_dir == new_dir):
		alpha = alpha/2
	[initial_value,fn,init_dir] = [new_value, new_fn, new_dir]
	if c > 50:
		print('cannot optimize')
		break
print('final value of z-spread =', round(new_value*100,2), '%')


[0.0, 5.0, 0.0, 5.0, 0.0, 105.0]
[0, 4.867685359633112, 0, 4.701649968703835, 0, 93.51169268807475] 103.0810280164117
final value of z-spread = 1.62 %
